In [ ]:
import folium
import pandas as pd

#create Folium map m
m = folium.Map(tiles = 'cartodbdark_matter')
folium.TileLayer("", name="None", attr="blank").add_to(m)
m.add_child(folium.map.LayerControl())


def filter(points, xLimLow, xLimHigh, yLimLow, yLimHigh):
    '''
    filters coordinates to be within the range of San Diego

    Params
    points- list of coordinates to be filtered
    xLimLow- the lowest x coordinate value to be accepted
    xLimHigh- the highest x coordinate value to be accepted
    yLimLow- the lowest y coordinate value to be accepted
    yLimHigh- the highest y coordinate value to be accepted

    return- list of points that are within the input coordinates boundary
    '''

    assert isinstance(points, list)
    assert isinstance(xLimLow, float)
    assert isinstance(xLimHigh, float)
    assert isinstance(yLimLow, float)
    assert isinstance(yLimHigh, float)

    filtered = []
    for i in points:
        if i[0] <= xLimHigh and i[0] >= xLimLow:
            if i[1] <= yLimHigh and i[1] >= yLimLow:
                filtered.append(i)
    
    return filtered


def add_to_map(response,opacities=0.1):
    '''
    uses coordinates to plot lines one by one onto a folium map

    Params
    response- API JSON request response containing all coordinates to form the line
    opacities- a list of the opacity to plot the line on the map

    Return
    m- folium map with lines plotted
    '''
    assert isinstance(response, dict)
    assert isinstance(opacities, float)

    coordinates = response['features'][0]['geometry']['coordinates']
    points = [(i[1], i[0]) for i in coordinates[0]]
    
    #filters the points to only include ones within San Diego
    points = filter(points, 32.55, 32.9, -117.3, -117.0)

    if len(points) > 0:
        folium.PolyLine(points, color = 'red',  weight=1, opacity=opacities).add_to(m)

        #sets the initial zoom boundaries
        df = pd.DataFrame(coordinates[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
        sw = df[['Lat', 'Lon']].min().values.tolist()
        ne = df[['Lat', 'Lon']].max().values.tolist()
        m.fit_bounds([sw, ne])

    return m


def create_map(csv_file):
    '''
    plots a csv file of coordinates as lines onto folium map m

    params
    csv_file- str location of csv file of coordinates
    '''
    
    assert isinstance(csv_file, str)
    assert csv_file[-4:] == '.csv'

    import ast
    data = pd.read_csv(csv_file)

    #scales the opacity relative to the volume of traffic in the dataset
    opacities = data['count'].values
    opacities = (opacities - opacities.min()) / (opacities.max() - opacities.min())
    opacities = opacities * 0.5 + 0.5

    #iterates through the data to plot each path one at a time
    for i in range(len(data)):
        data_i = data['coordinates'].iloc[i]
        json_data = ast.literal_eval(data_i)
        add_to_map(json_data,opacities = opacities[i])


create_map('../processed_datasets/sd_final.csv')

#calling m displays the map
m